# BLU02 - Learning Notebook - Data wrangling workflows - Part 3 of 3

In [1]:
import matplotlib.pyplot as plt

import pandas as pd
import os

# from category_encoders.ordinal import OrdinalEncoder
import category_encoders as ce
from category_encoders import OrdinalEncoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# 3 Advanced pipelines in scikit-learn

Remember our workflow diagram? Let's look at it again.

![data_transformation_workflow](./media/data_processing_workflow.png)

*Fig 1. - A standard workflow (again).*

Pandas, as amazing as it is, can only take us so far.

There, beyond the known universe, lies **modeling**.

Where we are at this point:
* We are to perform transformations on data, setting up robust pipelines using nothing but Pandas
* We can combine different dataframes, to enrich our datasets or generate new ones.

Thus, here we are, modeling lying ahead of us. What's exactly new about modeling though?

We will be using the same dataset, but this time we will create a train-test split, as we would do before modeling.

In [2]:
works = pd.read_csv('./data/works.csv')
X_train, X_test = train_test_split(works)

## 3.1 How is modeling different from transformation

In Pandas, we merely transformed the original dataframe into a new one.

But sometimes, this isn't possible. Let's start with an example: encoding categorical variables.

Remember: we need to perform the same transformations on train and test data (and whatever data comes next).

In [3]:
def transform_data(df):
    df = df.copy()
    df = (df.pipe(remove_intervals)
            .pipe(label_encoder, 'ComposerName'))
    return df


def remove_intervals(df):
    df = df.copy()
    mask = df['Interval'].isnull()
    df = (df.loc[mask, :]
            .drop(columns='Interval'))
    return df
    

def label_encoder(df, column):
    df = df.copy()
    df[column + 'Encoded'] = df[column].astype('category').cat.codes
    return df


X_train_ = transform_data(X_train)

train_alban_berg = X_train_['ComposerName'] == 'Berg,  Alban'
(X_train_.loc[train_alban_berg, ['ComposerName', 'ComposerNameEncoded']]
         .drop_duplicates())

,ComposerName,ComposerNameEncoded
35298,"Berg, Alban",176


All is good. We removed the intermissions (just like we did previously), and we transformed the original dataframe.

For convenience, we are keeping only the `ComposerName` and `ComposerNameEncoded` columns and removing duplicates.

Let's do the same to the test data.

In [4]:
X_test_ = transform_data(X_test)

test_alban_berg = X_test_['ComposerName'] == 'Berg,  Alban'
(X_test_.loc[test_alban_berg, ['ComposerName', 'ComposerNameEncoded']]
        .drop_duplicates())

,ComposerName,ComposerNameEncoded
23017,"Berg, Alban",103


Do you see the problem? The same `ComposerName` can (and will, in all probability) get different encodings.

This problem is significant, as it would lead us to make wrong predictions!!

There are other cases in which problems arise. For instance, when replacing missing values with the mean:
* You are supposed to compute the mean on the training set and use it to transform both train and test sets
* Otherwise, you end up underestimating your correct generalization error.

This particular unit is not about modeling at a conceptual level, but you get the point: 
* Somehow, you need to fit the transformer on your training data first (e.g., define the encodings, compute the means)
* Transform both train and test sets (and any data that might come in, really) using the pre-fitted transformers.

These transformations are more like modeling. In fact, all of this *is* modeling and part of your model. 

How do we solve this? **We need transformers that are more like models.**

## 3.2 Meet the sklearn-like transformers

There are at three fundamental verbs in scikit-learn and sklearn-like libraries:
* `.fit()`
* `.transform()`
* `.predict()`.

You are already familiar with `.fit()` and `.predict()`, from the Bootcamp and the Hackathon #1.  We use them to train models and make predictions.

Here, we will explore a new combo: `.fit()` and `.transform()`. This is how it works.

![sklearn_like_transformation_pipeline](./media/sklearn_like_transformation_pipeline.png)

*Fig 2. - A data pipeline with consistent transformers, fitted on the training set.*

In short, we fit a transformer on the training data and use to transform the training data.

We will, however, return the transformer so we can use it to transform new, incoming data as well. Confusing? Perhaps.

Time to get practical: meet the `categorical_encoders`, a set of transformers for encoding categorical variables.

In [5]:
encoder = OrdinalEncoder(cols=['ComposerName'])
X_train_ = encoder.fit_transform(X_train)
X_train_.head()

,GUID,ProgramID,WorkID,MovementID,ComposerName,WorkTitle,Movement,ConductorName,Interval
44885,81834875-362d-4277-81da-bfbc91676310,11588,52644,NaN,1,PRELUDE A L'APRES-MIDI D'UN FAUNE (PRELUDE TO ...,NaN,"Kostelanetz, Andre",NaN
30907,2937342c-ea68-4c1d-9ca1-377bfbf2d4ef,12687,0,NaN,2,NaN,NaN,NaN,Intermission
45484,df27b136-90de-4137-9d8f-ba5f86e1ef73,1968,52437,4.0,3,"SYMPHONY NO. 3 IN E FLAT MAJOR, OP. 55 (EROICA)",Finale: Allegro molto - Poco Andante - Presto,"Szell, George",NaN
25962,862827f2-d3a4-4eb2-b1ea-9d83245dd061,9789,0,NaN,2,NaN,NaN,NaN,Intermission
27886,9d575b43-ff50-4439-8c1f-c1cf907d2595,10523,10667,NaN,4,"SERENADE FOR STRING ORCHESTRA, OP. 23",NaN,"Damrosch, Frank",NaN


We can now use transform our test set.

In [18]:
X_test_ = encoder.fit_transform(X_test)
X_test_.head()

,GUID,ProgramID,WorkID,MovementID,ComposerName,WorkTitle,Movement,ConductorName,Interval
48032,c8db5013-aac5-46cc-ab91-8f5e65852d67,2242,52671,2.0,5,"COQ D'OR, LE: SUITE (ARR. Steinberg)",King Dodon and the Queen of Shemakha,"Kostelanetz, Andre",NaN
28063,0515f085-6a05-41af-8a46-2b760e02e831,377,9194,1.0,74,"BARTERED BRIDE, THE","Overture (""Overture to a Comedy"")","Mahler, Gustav",NaN
73011,ecfc9443-8dc0-4c6f-a36a-4c337a4c63af,9137,1898,2.0,4,"SALOME, OP. 54","Final Scene: ""Ah, du wolltest mich nicht deine...","Mehta, Zubin",NaN
56383,11abaa25-c4d4-4da2-adc9-6c219c0be140,10810,1998,NaN,73,"GOLD AND SILVER WALTZ, THE",NaN,"Stolz, Robert",NaN
7099,903907e7-361a-4d1a-8598-e551d9c1dd9e,11812,52193,NaN,666,IL MATRIMONIO SEGRETO,NaN,"Tzincoca, Rémus",NaN


Let's re-do our transformer functions so that they can either fit a transformer or accept a pre-fitted one.

We have to change our `label_encoder()` first to incorporate this logic. Then we need to adapt `transform_data()`.

In [19]:
def transform_data(df, encoder=None):
    df = df.copy()
    df, encoder = (df.pipe(remove_intervals)
                     .pipe(label_encoder, 'ComposerName', encoder))
    
    return df, encoder

def remove_intervals(df):
    df = df.copy()
    mask = df['Interval'].isnull()
    df = (df.loc[mask, :]
            .drop(columns='Interval'))
    return df


def label_encoder(df, columns, encoder=None):
    if not encoder:
        encoder = OrdinalEncoder(cols=[columns])
        encoder.fit(df)
        
    preview_encodings(encoder)

    df = df.copy()
    df = encoder.transform(df)

    return df, encoder

    
def preview_encodings(encoder):
    encodings = encoder.category_mapping[0]['mapping'][:4]
    print('Encodings: {}'.format(encodings))
    return None

X_train_, encoder = transform_data(X_train)

Encodings: [('Rodgers,  Richard', 1), ('Zhang,  Yijun', 2), ('Strauss,  Richard', 3), ('Rimsky-Korsakov,  Nikolai', 4)]


In the code above, we changed our functions so that they can receive an encoder. 

Otherwise, they fit and return the new one for re-use.

From a consistency standpoint, things should be looking good. Nonetheless, you are previewing the encoder as a sanity check.

In [20]:
X_test_ = transform_data(X_test, encoder=encoder)

Encodings: [('Rodgers,  Richard', 1), ('Zhang,  Yijun', 2), ('Strauss,  Richard', 3), ('Rimsky-Korsakov,  Nikolai', 4)]


What kind of transformations do you need to perform this way? Some widespread ones are:
* Encoding (as we've seen)
* Scaling
* Vectorization (you will learn about this in the next specialization!)
* Missing data imputation.

Now, this changes things (right?):
* We lose the ability to do method chaining, as we have to return encodings as intermediate outputs
* We need to segregate pipelines for training (fit and transform) and test (transform), which adds complexity and it's error-prone.

Because we will perform the same transformations on all datasets, storing all the correct steps is critical for reproducibility and consistency.

It turns out, scikit-learn provides us with a distinctive take on pipelines, to wrap all of this in a single META-TRANSFORMER.

![megazord](./media/megazord.png)

*Fig 3. - A meta-transformer in practice.*

Meet the Megazord.

## 3.3 Pipelines

The sklearn's `Pipelines` provide a higher level of abstraction than the individual building blocks.

Let's tie together all these sequential transformers and run `Megazord.fit()` and `Megazord.transform()` on the whole thing.

That would make managing our code much easier, right? Let's do it:
* We want to replace the missing values with the mode
* We want to one-hot-encode all categorical variables.

First things first, some Pandas magic: let's drop the ID columns and exclude the intervals.

In [21]:
def prepare_data(df):
    df = df.copy()
    df = (df.pipe(drop_id_fields)
            .pipe(remove_intervals)
            .drop_duplicates())
    return df

def drop_id_fields(df):
    columns = ['GUID', 'ProgramID', 'WorkID', 'MovementID']
    df = df.copy()
    df = df.drop(columns=columns)
    return df

X_train_ = prepare_data(X_train)
X_train_.head()

,ComposerName,WorkTitle,Movement,ConductorName
64137,"Rodgers, Richard",ALLEGRO,"""You Are Never Away""","Smallens, Alexander"
39076,"Zhang, Yijun",DREAMING SILK ROAD,NaN,"Maazel, Lorin"
67962,"Strauss, Richard",DEATH AND TRANSFIGURATION (TOD UND VERKLARUNG)...,NaN,"Toscanini, Arturo"
61807,"Rimsky-Korsakov, Nikolai","CAPRICCIO ESPAGNOL, OP. 34",NaN,"Stransky, Josef"
70732,"Schubert, Franz","ERLKONIG [ERLKÖNIG], D.328 (PIANO) (ARR. Liszt)",NaN,NaN


In [22]:
X_test_ = prepare_data(X_test)
X_test_.head()

,ComposerName,WorkTitle,Movement,ConductorName
48032,"Rimsky-Korsakov, Nikolai","COQ D'OR, LE: SUITE (ARR. Steinberg)",King Dodon and the Queen of Shemakha,"Kostelanetz, Andre"
28063,"Smetana, Bedrich","BARTERED BRIDE, THE","Overture (""Overture to a Comedy"")","Mahler, Gustav"
73011,"Strauss, Richard","SALOME, OP. 54","Final Scene: ""Ah, du wolltest mich nicht deine...","Mehta, Zubin"
56383,"Lehar [Lehár], Franz","GOLD AND SILVER WALTZ, THE",NaN,"Stolz, Robert"
7099,"Cimarosa, Domenico",IL MATRIMONIO SEGRETO,NaN,"Tzincoca, Rémus"


Here's how we are going to do:
1. Create an `Imputer` that replace missing values with the mode of the column
2. Use said `Imputer` to impute the missing values
3. Ordinal encode all the categorical features.

All in one, single, amazing, Megazord.

## 3.4 Custom transformers

We can build own custom transformers, for as long as they follow the usual blueprint:
* Implement `Transformer.fit()`
* And `Transformer.transform()`.

All scikit-learn estimators have `get_params()` and `set_params` functions. 

The easiest way to implement these functions sensibly is to inherit from `sklearn.base.BaseEstimator`, as we're doing.

And `Pipeline` compatibility requires a `fit_transform()` method that we are inheriting from `sklearn.base.TransformerMixin`.

In [ ]:
class FeatureMultiplier(BaseEstimator, TransformerMixin):
    def __init__(self, some_parameter):
        self.some_parameter = some_parameter

    def fit(self, X, y=None):
        # Fit the transformer and store it.
        return self
        
    def transform(self, X):
        # Transform X.
        return X

Also, we may want our transformer to accept some parameters.

That's we are doing when we include `some_parameter` in the `__init__`.

Back to our transformers. Our blueprint:
* We want the estimator to be able to take a `strategy` parameter, although we will support only the mode
* Fitting requires taking the mode of each column and storing it
* Transform implies replacing missing values with the given column modes.

How are we going to compute the modes? Pandas, as always, provides a convenient `.mode()` method.

In [23]:
X_train_.mode()

,ComposerName,WorkTitle,Movement,ConductorName
0,"Beethoven, Ludwig van",MESSIAH,Overture,"Damrosch, Walter"


To be able to use indexing we will use `df.squeeze()`, a convenient method to transform our dataframe into a `pd.Series`.

In [24]:
X_train.mode().squeeze()

GUID             884c64d6-1768-4cf1-85f1-0ac2f79bbe5c
ProgramID                                       10608
WorkID                                              0
MovementID                                          1
ComposerName                         Wagner,  Richard
WorkTitle                  MEISTERSINGER, DIE, WWV 96
Movement                                     Overture
ConductorName                        Damrosch, Walter
Interval                                 Intermission
Name: 0, dtype: object

We have everything we need.

In [25]:
class CategoryImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy=None):
        if strategy:
            self.strategy = strategy
        else:
            self.strategy = 'most_frequent'

    def fit(self, X, y=None):
        if self.strategy == 'most_frequent':
            self.fills = X.mode(axis=0).squeeze()
            return self
        else:
            return 'Strategy not supported.'

    def transform(self, X):
        return pd.DataFrame(X).fillna(self.fills)


imputer = CategoryImputer()
X_train_ = imputer.fit_transform(X_train_)
X_train_.head()

,ComposerName,WorkTitle,Movement,ConductorName
64137,"Rodgers, Richard",ALLEGRO,"""You Are Never Away""","Smallens, Alexander"
39076,"Zhang, Yijun",DREAMING SILK ROAD,Overture,"Maazel, Lorin"
67962,"Strauss, Richard",DEATH AND TRANSFIGURATION (TOD UND VERKLARUNG)...,Overture,"Toscanini, Arturo"
61807,"Rimsky-Korsakov, Nikolai","CAPRICCIO ESPAGNOL, OP. 34",Overture,"Stransky, Josef"
70732,"Schubert, Franz","ERLKONIG [ERLKÖNIG], D.328 (PIANO) (ARR. Liszt)",Overture,"Damrosch, Walter"


In [26]:
X_train_.isnull().sum()

ComposerName     0
WorkTitle        0
Movement         0
ConductorName    0
dtype: int64

There we go! What about the test set?

In [27]:
X_test_ = imputer.transform(X_test_)
X_test_.isnull().sum()

ComposerName     0
WorkTitle        0
Movement         0
ConductorName    0
dtype: int64

Victory awaits!

## 3.5 Everything together

Now, we want to fill in missing values and use one-hot-encoding (remember?), all at the same time.

We are reaching our destination!

In [28]:
megazord = Pipeline([('fill_na', CategoryImputer(strategy='most_frequent')),
                     ('encode', OrdinalEncoder())])

X_train_ = megazord.fit_transform(X_train)
X_test_ = megazord.transform(X_test)

This way we abstract all the logic of passing transformers around.

Now, can we throw a model in there? Perhaps we can.

(But we shouldn't, in a way. Please note that we are exemplifying data wrangling workflows.)

In [29]:
megazord = Pipeline([('fill_na', CategoryImputer(strategy='most_frequent')),
                     ('encode', OrdinalEncoder()),
                     ('k_means', KMeans())])

megazord.fit(X_train_)
megazord.predict(X_test_)

array([3, 2, 4, ..., 6, 0, 3])

For the sake of simplicity, we are encoding categorical variables as if they were ordinal, instead of using one-hot-encoding, as recommended.

Take this for what it is: an example on how to build end-to-end pipelines for modeling in scikit-learn.